# Подбор модели с использованием AutoML

Подбор пайплайнов с помощью AutoML пакета [GAMA](https://openml-labs.github.io/gama/master)  

Тестировались разные базовые методы поиска оптимального пайплайна, включенные в GAMA:

- _Asynchronous Evolutionary Algorithm_: (генетический поиск) Evolve a population of machine learning pipelines, drawing new machine learning pipelines from the best of the population.

- _Asynchronous Successive Halving Algorithm_: A bandit-based approach where many machine learning pipelines iteratively get evaluated and eliminated on bigger fractions of the data. 

Разные тесты проводились с разным лимитом времени, от 3 мин до 3 часов


In [7]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

NOTEBOOK_ID = '2'  # в выходных данных будет указан этот id
RANDOM_SEED = 0  # для воспроизводимости результатов
ROOT_PATH = Path('..')  # путь к корню проекта

In [8]:
import datetime as dt
import logging

import pandas as pd

from gama import GamaRegressor
from gama.search_methods import AsynchronousSuccessiveHalving

In [9]:
def select_pipe(
        X_train,
        y_train,
        *,
        search=None,
        max_total_time=3600,
        scoring=('neg_mean_squared_error', 'r2'),
        random_state=RANDOM_SEED,
        regularize_length=True,
        preset='simple',
        n_jobs=1,
        verbose=logging.WARNING,
    ):
    """
    Находим лучшую модель-регрессор для заданного набора данных.
    """
    gama = GamaRegressor(
        search=search,
        max_total_time=max_total_time, 
        scoring=scoring,                      
        random_state=random_state, 
        regularize_length=regularize_length,
        # store="logs",
        preset=preset,
        n_jobs=n_jobs,
        verbosity=verbose
    )
    
    delta = max_total_time / 60
    t0 = dt.datetime.now()
    eta = t0 + dt.timedelta(seconds=max_total_time)
    def to_str(t):
        return t.strftime('%Y-%m-%d %H:%M')

    print(f'{to_str(t0)} Подбор модели завершится приблизительно за {delta} мин, в {to_str(eta)}')
    gama.fit(X_train, y_train)
    t1 = dt.datetime.now()
    print(f'{to_str(t1)} Модель подобрана, score={gama.score(X_train, y_train)}')

    return gama

## Подбор с AsynchronousSuccessiveHalving 

In [10]:
# Загружаем обучающие и тестовые данные, уже очищенные от неверных значений

X_train = pd.read_csv(ROOT_PATH / 'data/processed/train_X__pipe_main_fittedon_train_1.csv')
y_train = pd.read_csv(ROOT_PATH / 'data/processed/train_y_1.csv')

print('Обучающая выборка:', X_train.shape)

X_test = pd.read_csv(ROOT_PATH / 'data/processed/test_X__pipe_main_fittedon_train_1.csv')
y_test = pd.read_csv(ROOT_PATH / 'data/processed/test_y_1.csv')

print('Тестовая выборка: ', X_test.shape)

Обучающая выборка: (1164, 67)
Тестовая выборка:  (292, 67)


In [11]:
# Подбираем модель

MAX_TOTAL_TIME = 60 * 60  # sec
N_JOBS = 6
MODEL_NAME = f'gama_ASH_{MAX_TOTAL_TIME}s'

gama = select_pipe(
    X_train,
    y_train,
    search=AsynchronousSuccessiveHalving(),
    max_total_time=MAX_TOTAL_TIME,
    n_jobs=N_JOBS,
)

# сохраняем подобранную модель в файл
file_py = ROOT_PATH / f'models/predictors/{MODEL_NAME}_{NOTEBOOK_ID}.py'
gama.export_script(file_py)
print(f'Подобранная модель сохранена в файл: {file_py}')

2023-11-21 08:39 Подбор модели завершится приблизительно за 60.0 мин, в 2023-11-21 09:39
2023-11-21 09:33 Модель подобрана, score=0.0
Подобранная модель сохранена в файл: ..\models\predictors\gama_ASH_3600s_2.py


In [12]:
# Если ничего не напечаталось - то не дошли сюда, и где-то в ноутбуке ошибка
import datetime as dt
print('Ноутбук завершен:', dt.datetime.now())

Ноутбук завершен: 2023-11-21 09:33:58.376988
